# Optymalizacja masy

In [2]:
using JuMP
using CSV
using DataFrames
using AmplNLWriter

### Prepare input data
data_mass = dropmissing(CSV.read("input_data/plecak.csv"))
data_plan = CSV.read("input_data/plan.csv")

plan_summary = by(data_plan, [:Przedmiot], plan_summary -> size(plan_summary, 1))
mass_summary = by(data_mass, [:Przedmiot], mass_summary -> sum(mass_summary[:Waga]))

przybory_mass = mass_summary[mass_summary[:Przedmiot] .== "Przybory", :][:x1] / 1e3

mass_summary = mass_summary[mass_summary[:Przedmiot] .!= "Przybory", :]

dict_plan = Dict(zip(plan_summary[:Przedmiot], plan_summary[:x1]))
dict_mass = Dict(zip(mass_summary[:Przedmiot], mass_summary[:x1]))

data_in= DataFrame(Przedmiot = String[],
    Sztuk = Int64[],
    Waga = Float64[])

for row in 1:size(plan_summary, 1)
    course = plan_summary[:Przedmiot][row]
    push!(data_in,[course, get(dict_plan, course, 0), get(dict_mass, course, 0) / 1e3])
end
println(data_in)
### set parameters
course_no = size(data_in,1)
max_course_occurrences = maximum(data_in[:Sztuk])
days = 5

# avg mass for comparison
avg_mass = sum(data_in[:Sztuk].*data_in[:Waga]) / days + przybory_mass

# objective scaling factor - workaround to set solution tolerance
objective_scaling_factor = 20e-3
data_in[:Waga] = data_in[:Waga] * objective_scaling_factor

# dictionaries for results
course_map = Dict(zip(1:course_no, data_in[:Przedmiot]))
mass_map = Dict(zip(1:course_no, data_in[:Waga]))

# maximum number of a course occurences per day
max_occurences_per_day  = 2
if max_occurences_per_day < max_course_occurrences
    max_course_occurrences = max_occurences_per_day
end



print("Masa przyborów: ")
println(przybory_mass[1])

11×3 DataFrames.DataFrame
│ Row │ Przedmiot   │ Sztuk │ Waga  │
├─────┼─────────────┼───────┼───────┤
│ 1   │ Matematyka  │ 4     │ 0.697 │
│ 2   │ Angielski   │ 3     │ 0.715 │
│ 3   │ Polski      │ 5     │ 1.676 │
│ 4   │ WF          │ 3     │ 0.297 │
│ 5   │ Religia     │ 2     │ 0.568 │
│ 6   │ Przyroda    │ 2     │ 1.214 │
│ 7   │ Technika    │ 1     │ 0.756 │
│ 8   │ Informatyka │ 1     │ 0.159 │
│ 9   │ Plastyka    │ 1     │ 0.167 │
│ 10  │ Historia    │ 1     │ 0.484 │
│ 11  │ Muzyka      │ 1     │ 0.36  │
Masa przyborów: 1.415


In [3]:
### Opt start, define model, variables, constraints and objective
# Solver engine can be found on from https://ampl.com/products/solvers/open-source/ 
solver_couenne="couenne-win64/couenne.exe"
mdl=Model(solver=AmplNLSolver(solver_couenne,["tol=1"]))

@variable(mdl, x[1:days,1:course_no, 1:max_course_occurrences], Bin, start=1)
@variable(mdl, day[1:days])

for d in 1:days
   @constraint(mdl, day[d] == sum(sum(x[d,c,m]*data_in[:Waga][c] 
                for c in 1:course_no) 
                for m in 1:max_course_occurrences))
    
    for c in 1:course_no
            @constraint(mdl, sum(x[d,c,m]*m 
                    for m in 1:max_course_occurrences) <= max_course_occurrences)
    end
end

for c in 1:course_no
    @constraint(mdl, sum(sum(x[d,c,m]*m 
                for d in 1:days) 
                for m in 1:max_course_occurrences) == data_in[:Sztuk][c])
end

@NLobjective(mdl,Min, sum((day[d])^2 for d in 1:days))

print(mdl)

Min day[1] ^ 2.0 + day[2] ^ 2.0 + day[3] ^ 2.0 + day[4] ^ 2.0 + day[5] ^ 2.0
Subject to
 day[1] - 0.01394 x[1,1,1] - 0.0143 x[1,2,1] - 0.03352 x[1,3,1] - 0.00594 x[1,4,1] - 0.011359999999999999 x[1,5,1] - 0.02428 x[1,6,1] - 0.01512 x[1,7,1] - 0.00318 x[1,8,1] - 0.00334 x[1,9,1] - 0.00968 x[1,10,1] - 0.0072 x[1,11,1] - 0.01394 x[1,1,2] - 0.0143 x[1,2,2] - 0.03352 x[1,3,2] - 0.00594 x[1,4,2] - 0.011359999999999999 x[1,5,2] - 0.02428 x[1,6,2] - 0.01512 x[1,7,2] - 0.00318 x[1,8,2] - 0.00334 x[1,9,2] - 0.00968 x[1,10,2] - 0.0072 x[1,11,2] == 0
 x[1,1,1] + 2 x[1,1,2] <= 2
 x[1,2,1] + 2 x[1,2,2] <= 2
 x[1,3,1] + 2 x[1,3,2] <= 2
 x[1,4,1] + 2 x[1,4,2] <= 2
 x[1,5,1] + 2 x[1,5,2] <= 2
 x[1,6,1] + 2 x[1,6,2] <= 2
 x[1,7,1] + 2 x[1,7,2] <= 2
 x[1,8,1] + 2 x[1,8,2] <= 2
 x[1,9,1] + 2 x[1,9,2] <= 2
 x[1,10,1] + 2 x[1,10,2] <= 2
 x[1,11,1] + 2 x[1,11,2] <= 2
 day[2] - 0.01394 x[2,1,1] - 0.0143 x[2,2,1] - 0.03352 x[2,3,1] - 0.00594 x[2,4,1] - 0.011359999999999999 x[2,5,1] - 0.02428 x[2,6,1] - 0.01512

In [4]:
# solve
status = solve(mdl)

Couenne 0.5.6 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: tol=1

ANALYSIS TEST: NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 0.0094784548        7 0.009
Loaded instance "C:\Users\bialekj\.julia\v0.6\AmplNLWriter\.solverdata\jl_1F4C.tmp.nl"
Constraints:           71
Variables:            115 (110 integer)
Auxiliaries:           22 (16 integer)

Coin0506I Presolve 71 (-26) rows, 95 (-42) columns and 285 (-86) elements
Clp0006I 0  Obj 0 Primal inf 9.2218132 (11)
Clp0006I 39  Obj 1.124674e-017 Primal inf 1.2576454 (8)
Clp0006I 70  Obj 0.00771767
Clp0000I Optimal - objective value 0.00771767
Clp0032I Optimal objective 0.00771767 - 70 iterations time 0.002, Presolve 0.00
Clp0000I Optimal - objective value 0.00771767
NLP Heuristic: time limit reached.
Clp0000I Optimal - objective value 0.007

Cbc0010I After 5200 nodes, 1394 on tree, 0.011822675 best solution, best possible 0.010243968 (5.60 seconds)
Cbc0010I After 5300 nodes, 1424 on tree, 0.011822675 best solution, best possible 0.010243968 (5.65 seconds)
Cbc0010I After 5400 nodes, 1447 on tree, 0.011822675 best solution, best possible 0.010243968 (5.72 seconds)
Cbc0010I After 5500 nodes, 1479 on tree, 0.011822675 best solution, best possible 0.010243968 (5.78 seconds)
Cbc0010I After 5600 nodes, 1507 on tree, 0.011822675 best solution, best possible 0.010243968 (5.85 seconds)
Cbc0010I After 5700 nodes, 1536 on tree, 0.011822675 best solution, best possible 0.010243968 (5.90 seconds)
Cbc0010I After 5800 nodes, 1561 on tree, 0.011822675 best solution, best possible 0.010243968 (5.97 seconds)
Cbc0010I After 5900 nodes, 1580 on tree, 0.011822675 best solution, best possible 0.010243968 (6.03 seconds)
Cbc0010I After 6000 nodes, 1607 on tree, 0.011822675 best solution, best possible 0.010243968 (6.10 seconds)
Cbc0010I After 6100

Cbc0010I After 12700 nodes, 3227 on tree, 0.011822675 best solution, best possible 0.010243968 (10.71 seconds)
Cbc0010I After 12800 nodes, 3251 on tree, 0.011822675 best solution, best possible 0.010243968 (10.77 seconds)
Cbc0010I After 12900 nodes, 3277 on tree, 0.011822675 best solution, best possible 0.010243968 (10.86 seconds)
Cbc0010I After 13000 nodes, 3295 on tree, 0.011822675 best solution, best possible 0.010243968 (10.92 seconds)
Cbc0010I After 13100 nodes, 3318 on tree, 0.011822675 best solution, best possible 0.010243968 (10.99 seconds)
Cbc0010I After 13200 nodes, 3342 on tree, 0.011822675 best solution, best possible 0.010243968 (11.07 seconds)
Cbc0010I After 13300 nodes, 3375 on tree, 0.011822675 best solution, best possible 0.010243968 (11.14 seconds)
Cbc0010I After 13400 nodes, 3402 on tree, 0.011822675 best solution, best possible 0.010243968 (11.20 seconds)
Cbc0010I After 13500 nodes, 3430 on tree, 0.011822675 best solution, best possible 0.010243968 (11.26 seconds)
C

Cbc0010I After 20100 nodes, 4452 on tree, 0.011822675 best solution, best possible 0.010248579 (15.63 seconds)
Cbc0010I After 20200 nodes, 4455 on tree, 0.011822675 best solution, best possible 0.01024895 (15.69 seconds)
Cbc0010I After 20300 nodes, 4459 on tree, 0.011822675 best solution, best possible 0.010249316 (15.75 seconds)
Cbc0010I After 20400 nodes, 4471 on tree, 0.011822675 best solution, best possible 0.010249479 (15.82 seconds)
Cbc0010I After 20500 nodes, 4491 on tree, 0.011822675 best solution, best possible 0.01024948 (15.90 seconds)
Cbc0010I After 20600 nodes, 4503 on tree, 0.011822675 best solution, best possible 0.010249563 (15.97 seconds)
Cbc0010I After 20700 nodes, 4517 on tree, 0.011822675 best solution, best possible 0.010249632 (16.03 seconds)
Cbc0010I After 20800 nodes, 4537 on tree, 0.011822675 best solution, best possible 0.010249686 (16.09 seconds)
Cbc0010I After 20900 nodes, 4538 on tree, 0.011822675 best solution, best possible 0.01025002 (16.15 seconds)
Cbc0

Cbc0010I After 27500 nodes, 5114 on tree, 0.011822675 best solution, best possible 0.010285471 (20.97 seconds)
Cbc0010I After 27600 nodes, 5108 on tree, 0.011822675 best solution, best possible 0.010287222 (21.04 seconds)
Cbc0010I After 27700 nodes, 5105 on tree, 0.011822675 best solution, best possible 0.010288745 (21.12 seconds)
Cbc0010I After 27800 nodes, 5099 on tree, 0.011822675 best solution, best possible 0.010291556 (21.18 seconds)
Cbc0010I After 27900 nodes, 5087 on tree, 0.011822675 best solution, best possible 0.01029479 (21.26 seconds)
Cbc0010I After 28000 nodes, 5081 on tree, 0.011822675 best solution, best possible 0.010297918 (21.33 seconds)
Cbc0010I After 28100 nodes, 5085 on tree, 0.011822675 best solution, best possible 0.010298675 (21.40 seconds)
Cbc0010I After 28200 nodes, 5086 on tree, 0.011822675 best solution, best possible 0.010300187 (21.47 seconds)
Cbc0010I After 28300 nodes, 5087 on tree, 0.011822675 best solution, best possible 0.010303093 (21.54 seconds)
Cb

...

Cbc0010I After 72100 nodes, 6729 on tree, 0.011822675 best solution, best possible 0.011799412 (52.25 seconds)
Cbc0010I After 72200 nodes, 6716 on tree, 0.011822675 best solution, best possible 0.011799698 (52.31 seconds)
Cbc0010I After 72300 nodes, 6717 on tree, 0.011822675 best solution, best possible 0.01179973 (52.38 seconds)
Cbc0010I After 72400 nodes, 6716 on tree, 0.011822675 best solution, best possible 0.011799869 (52.45 seconds)
Cbc0010I After 72500 nodes, 6717 on tree, 0.011822675 best solution, best possible 0.011799895 (52.51 seconds)
Cbc0010I After 72600 nodes, 6675 on tree, 0.011822675 best solution, best possible 0.011799916 (52.56 seconds)
Cbc0010I After 72700 nodes, 6630 on tree, 0.011822675 best solution, best possible 0.011800111 (52.61 seconds)
Cbc0010I After 72800 nodes, 6589 on tree, 0.011822675 best solution, best possible 0.011800248 (52.66 seconds)
Cbc0010I After 72900 nodes, 6563 on tree, 0.011822675 best solution, best possible 0.011800402 (52.72 seconds)
Cb

Cbc0010I After 79500 nodes, 5795 on tree, 0.011822675 best solution, best possible 0.011805367 (56.64 seconds)
Cbc0010I After 79600 nodes, 5794 on tree, 0.011822675 best solution, best possible 0.011805367 (56.70 seconds)
Cbc0010I After 79700 nodes, 5793 on tree, 0.011822675 best solution, best possible 0.011805367 (56.76 seconds)
Cbc0010I After 79800 nodes, 5793 on tree, 0.011822675 best solution, best possible 0.011805367 (56.81 seconds)
Cbc0010I After 79900 nodes, 5790 on tree, 0.011822675 best solution, best possible 0.011805367 (56.87 seconds)
Cbc0010I After 80000 nodes, 5789 on tree, 0.011822675 best solution, best possible 0.011805367 (56.93 seconds)
Cbc0010I After 80100 nodes, 5784 on tree, 0.011822675 best solution, best possible 0.011805367 (56.98 seconds)
Cbc0010I After 80200 nodes, 5781 on tree, 0.011822675 best solution, best possible 0.011805367 (57.04 seconds)
Cbc0010I After 80300 nodes, 5780 on tree, 0.011822675 best solution, best possible 0.011805367 (57.10 seconds)
C

Cbc0010I After 87000 nodes, 4879 on tree, 0.011822675 best solution, best possible 0.011809392 (61.47 seconds)
Cbc0010I After 87100 nodes, 4866 on tree, 0.011822675 best solution, best possible 0.011809392 (61.58 seconds)
Cbc0010I After 87200 nodes, 4855 on tree, 0.011822675 best solution, best possible 0.011809392 (61.66 seconds)
Cbc0010I After 87300 nodes, 4845 on tree, 0.011822675 best solution, best possible 0.011809392 (61.76 seconds)
Cbc0010I After 87400 nodes, 4834 on tree, 0.011822675 best solution, best possible 0.011809392 (61.83 seconds)
Cbc0010I After 87500 nodes, 4815 on tree, 0.011822675 best solution, best possible 0.011809423 (61.90 seconds)
Cbc0010I After 87600 nodes, 4789 on tree, 0.011822675 best solution, best possible 0.011809491 (61.96 seconds)
Cbc0010I After 87700 nodes, 4773 on tree, 0.011822675 best solution, best possible 0.011809532 (62.03 seconds)
Cbc0010I After 87800 nodes, 4753 on tree, 0.011822675 best solution, best possible 0.011809561 (62.10 seconds)
C

Cbc0010I After 95400 nodes, 2787 on tree, 0.011822675 best solution, best possible 0.011810616 (66.44 seconds)
Cbc0010I After 95500 nodes, 2774 on tree, 0.011822675 best solution, best possible 0.011810616 (66.50 seconds)
Cbc0010I After 95600 nodes, 2764 on tree, 0.011822675 best solution, best possible 0.011810616 (66.55 seconds)
Cbc0010I After 95700 nodes, 2751 on tree, 0.011822675 best solution, best possible 0.011810616 (66.61 seconds)
Cbc0010I After 95800 nodes, 2734 on tree, 0.011822675 best solution, best possible 0.011810616 (66.66 seconds)
Cbc0010I After 95900 nodes, 2717 on tree, 0.011822675 best solution, best possible 0.011810618 (66.72 seconds)
Cbc0010I After 96000 nodes, 2678 on tree, 0.011822675 best solution, best possible 0.011810641 (66.77 seconds)
Cbc0010I After 96100 nodes, 2628 on tree, 0.011822675 best solution, best possible 0.011810648 (66.82 seconds)
Cbc0010I After 96200 nodes, 2578 on tree, 0.011822675 best solution, best possible 0.011810648 (66.87 seconds)
C

:Optimal

In [6]:
### print and save results
day_of_week=Dict(zip([1, 2, 3, 4, 5], ["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"]))

plan = DataFrame([String,String,String,String,String],
    [:Poniedzialek,:Wtorek,:Sroda,:Czwartek,:Piatek], 10)
sum_week = 0

for d in 1:days
    i=1
    plan[i,d] = string(getvalue(day[d]) / objective_scaling_factor + przybory_mass)
    sum_week += getvalue(day[d]) / objective_scaling_factor + przybory_mass
    i+=1
    for c in 1:course_no
        for m in 1:max_course_occurrences
            value = getvalue(x[d,c,m])*m
            if abs(value-0.0)>1e-3              
                for repeat in 1:m
                    plan[i,d] = get(course_map, c, "brak kursu w slowniku")
                    i+=1
                end
            end
        end
    end
    for rest in i:10
         plan[rest,d] = ""
    end
end

max_mass = maximum(getvalue(day)) / objective_scaling_factor + przybory_mass


println("masa maxymalna")
println(max_mass)
println("masa sumaryczna")
println(sum_week)
println("masa srednia bez optim")
println(avg_mass)
print(plan)

CSV.write("results/plan_wynik_min.csv",plan)

masa maxymalna
[3.907]
masa sumaryczna
[19.229]
masa srednia bez optim
[5.3538]
10×5 DataFrames.DataFrame
│ Row │ Poniedzialek │ Wtorek    │ Sroda      │ Czwartek   │ Piatek   │
├─────┼──────────────┼───────────┼────────────┼────────────┼──────────┤
│ 1   │ [3.907]      │ [3.806]   │ [3.788]    │ [3.859]    │ [3.869]  │
│ 2   │ Polski       │ Angielski │ Matematyka │ Matematyka │ Przyroda │
│ 3   │ Polski       │ Polski    │ Matematyka │ Matematyka │ Przyroda │
│ 4   │ WF           │ Polski    │ Polski     │ Angielski  │ Technika │
│ 5   │ WF           │           │            │ Angielski  │ Historia │
│ 6   │ Informatyka  │           │            │ WF         │          │
│ 7   │ Muzyka       │           │            │ Religia    │          │
│ 8   │              │           │            │ Religia    │          │
│ 9   │              │           │            │ Plastyka   │          │
│ 10  │              │           │            │            │          │

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "results/plan_wynik_min.csv", 42, true, String["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"], 5, false, Val{false})